<a href="https://colab.research.google.com/github/hungkien05/notebook/blob/master/my_nmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    #/content/drive/MyDrive/Data/
    lines = open('train.%s' % (lang1), encoding='utf-8').\
        read().strip().split('\n')
    # tien xu ly: https://analyticsindiamag.com/how-to-create-a-vocabulary-builder-for-nlp-tasks/
    

    return lines

In [9]:
from torch.functional import Tensor
class Vocab:
  def __init__(self, lang, lines):
    self.lang=lang
    self.lines=lines
    self.words = set()
    #self.indices = []
    self.dictionary = dict()
    self.word_to_idx= dict()
    self.indices = torch.LongTensor()
    self.size=0

  def add_sentence(self,sentence):
    for word in sentence.split(' '):
      self.words.add(word)

  def build_vocab(self):
    for sentence in lines:
      self.add_sentence(sentence)
    self.word_to_idx = {word: i for i,word in enumerate(self.words)}
    self.indices = torch.LongTensor([self.word_to_idx[word] for word in self.words]) # create index tensor
    self.size = len(self.words)

    


In [10]:
lines = readLangs("en", "vi")
eng_vocab = Vocab("en", lines)
eng_vocab.build_vocab()
#eng_vocab.words


Reading lines...


In [16]:
embed = nn.Embedding(eng_vocab.size, 10)
print(embed(eng_vocab.indices))

tensor([[ 0.4974, -0.5363,  0.5842,  ..., -0.9518,  0.4767,  0.0030],
        [ 0.5484,  1.1578,  1.5077,  ..., -0.7099, -1.3904,  0.4672],
        [-0.7445,  0.1795,  0.6405,  ..., -0.0036,  0.0749, -2.1796],
        ...,
        [-1.0967,  0.3138,  0.3261,  ..., -0.0932, -0.3927,  0.5050],
        [-0.0393, -0.0626, -0.1809,  ..., -1.2488,  1.0228, -0.7464],
        [-1.0946, -0.5065, -0.9193,  ..., -0.6371,  0.6114, -0.1085]],
       grad_fn=<EmbeddingBackward0>)


In [3]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, batch_size):
        super(Encoder, self).__init__()
        self.n_layer=1
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.embed = nn.Embedding(input_dim, embed_dim)
        self.lstm = nn.LSTM(input_size=embed_dim, hidden_size=hidden_dim, batch_first=True)


    def forward(self, enc_input):
        tmp = self.embed(enc_input)
        h0 = torch.randn(self.n_layer, self.batch_size, self.hidden_dim)
        c0 = torch.randn(self.n_layer, self.batch_size, self.hidden_dim)
        encoder_output, (hn,cn)= self.lstm(tmp, h0, c0)
        return (hn,cn)


In [4]:
class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size):
        super(Decoder, self).__init__()
        self.input_dim= input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, batch_first=True)

    def forward(self, dec_input):
        h0 = torch.randn(self.n_layer, self.batch_size, self.hidden_dim)
        c0 = torch.randn(self.n_layer, self.batch_size, self.hidden_dim)
        dec_output, (hn,cn)= self.lstm(dec_input, h0, c0)
        return dec_output